In [13]:
import ray
import torch
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
from ray import train
from ray.air import session
from torch import nn

from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit import QuantumCircuit
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

from quantum_serverless.train.trainer import (
    QiskitScalingConfig,
    QiskitTorchTrainer,
    get_runtime_session,
)

In [14]:
INPUT_SIZE = 1
LAYER_SIZE = 2
OUTPUT_SIZE = 1
NUM_EPOCHS = 1

In [15]:
def create_qnn(session, layer_size):
    feature_map = ZZFeatureMap(layer_size)
    ansatz = RealAmplitudes(layer_size, reps=1)

    qc = QuantumCircuit(layer_size)
    qc.append(feature_map, range(layer_size))
    qc.append(ansatz, range(layer_size))

    qnn = EstimatorQNN(
        estimator=Estimator(session=session),
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True
    )
    return TorchConnector(qnn)

In [16]:
class NeuralNetwork(nn.Module):
    """Test neural network."""

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(LAYER_SIZE, OUTPUT_SIZE)

    def forward(self, input_tensor):
        """Forward pass."""
        return self.layer2(self.relu(self.layer1(input_tensor)))

In [17]:
class HybridQNN(nn.Module):
    """Test neural network."""

    def __init__(self, session):
        super().__init__()
        self.layer1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.relu = nn.ReLU()
        self.qnn = create_qnn(session, LAYER_SIZE)

    def forward(self, input_tensor):
        """Forward pass."""
        x = self.relu(self.layer1(input_tensor))
        x = self.qnn(x)
        return x

In [18]:
def train_loop(config):
    """Test training loop."""
    runtime_session = get_runtime_session(config)
    print("Session", runtime_session)

    is_qnn = config.get("is_qnn")
    dataset_shard = session.get_dataset_shard("train")
    loss_fn = nn.MSELoss()

    print("DS Shard: ", dataset_shard)

    if is_qnn:
        model = HybridQNN(runtime_session)
    else:
        model = NeuralNetwork()

    print("Model: ", model)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

    model = train.torch.prepare_model(model)

    for epoch in range(NUM_EPOCHS):
        print("Epoch: ", epoch)
        for batches in dataset_shard.iter_torch_batches(batch_size=1, dtypes=torch.float):
            inputs, labels = torch.unsqueeze(batches["x"], 1), batches["y"]
            print("inputs, labels: ", inputs, labels)
            output = model(inputs)
            print("output: ", output)
            loss = loss_fn(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"epoch: {epoch}, loss: {loss.item()}")

In [19]:
from tokens import Tokens
API_TOKEN = Tokens.API_TOKEN[0]
QiskitRuntimeService.save_account(channel="ibm_quantum",
                                  token=API_TOKEN,
                                  overwrite=True)

In [20]:
"""Tests trainer."""
train_dataset = ray.data.from_items(
    [{"x": x, "y": 2 * x + 1} for x in range(2)]
)

In [21]:
scaling_config = QiskitScalingConfig(num_workers=1, num_qubits=1, simulator=True)

In [22]:
runtime_service = QiskitRuntimeService(channel="ibm_quantum")
print("runtime service: ", runtime_service)

runtime service:  <QiskitRuntimeService>


In [23]:
trainer = QiskitTorchTrainer(
    train_loop_per_worker=train_loop,
    qiskit_runtime_service_account=runtime_service.active_account(),
    scaling_config=scaling_config,
    datasets={"train": train_dataset},
    train_loop_config={"is_qnn": True},
)

In [24]:
trainer.fit()

(RayTrainWorker pid=76949) 2023-01-26 15:42:19,519	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=1]


(RayTrainWorker pid=76949) Getting runtime session
(RayTrainWorker pid=76949) Backend name:  ibmq_qasm_simulator
(RayTrainWorker pid=76949) Session <qiskit_ibm_runtime.session.Session object at 0x7fb1a0452ac0>
(RayTrainWorker pid=76949) DS Shard:  Dataset(num_blocks=2, num_rows=2, schema={x: int64, y: int64})
(RayTrainWorker pid=76949) Model:  HybridQNN(
(RayTrainWorker pid=76949)   (layer1): Linear(in_features=1, out_features=2, bias=True)
(RayTrainWorker pid=76949)   (relu): ReLU()
(RayTrainWorker pid=76949)   (qnn): TorchConnector()
(RayTrainWorker pid=76949) )
(RayTrainWorker pid=76949) Epoch:  0
(RayTrainWorker pid=76949) inputs, labels:  tensor([[0.]]) tensor([1.])


(RayTrainWorker pid=76949) 2023-01-26 15:42:26,739	INFO train_loop_utils.py:270 -- Moving model to device: cpu
(RayTrainWorker pid=76949) runtime_job._start_websocket_client:WARNING:2023-01-26 15:42:52,151: An error occurred while streaming results from the server for job cf992lapm2vc8eqhgqjg:
(RayTrainWorker pid=76949) Traceback (most recent call last):
(RayTrainWorker pid=76949)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/runtime_job.py", line 454, in _start_websocket_client
(RayTrainWorker pid=76949)     self._ws_client.job_results()
(RayTrainWorker pid=76949)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/api/clients/runtime_ws.py", line 70, in job_results
(RayTrainWorker pid=76949)     self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
(RayTrainWorker pid=76949)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/api/clients/base.py", line 230, in s

(RayTrainWorker pid=76949) output:  tensor([[0.4675]], grad_fn=<_TorchNNFunctionBackward>)


(RayTrainWorker pid=76949) /opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
(RayTrainWorker pid=76949)   return F.mse_loss(input, target, reduction=self.reduction)
(RayTrainWorker pid=76949) runtime_job._start_websocket_client:WARNING:2023-01-26 15:43:22,259: An error occurred while streaming results from the server for job cf992rbrtkrl1nfrsrbg:
(RayTrainWorker pid=76949) Traceback (most recent call last):
(RayTrainWorker pid=76949)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/runtime_job.py", line 454, in _start_websocket_client
(RayTrainWorker pid=76949)     self._ws_client.job_results()
(RayTrainWorker pid=76949)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/

...

(RayTrainWorker pid=76949) epoch: 0, loss: 6.235008716583252


2023-01-26 15:44:18,435	ERROR checkpoint_manager.py:327 -- Result dict has no key: training_iteration. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['trial_id', 'experiment_id', 'date', 'timestamp', 'pid', 'hostname', 'node_ip', 'done']


Trial QiskitTorchTrainer_95998_00000 completed. Last result: 


2023-01-26 15:44:18,549	INFO tune.py:762 -- Total run time: 141.11 seconds (141.00 seconds for the tuning loop).


Result(metrics={'trial_id': '95998_00000', 'done': True}, error=None, log_dir=PosixPath('/Users/ept/ray_results/QiskitTorchTrainer_2023-01-26_15-41-57/QiskitTorchTrainer_95998_00000_0_2023-01-26_15-41-57'))